In [1]:
# ------------------------------
# Cost Calculation for Nodes
# ------------------------------
def calculate_cost(heuristics, condition, weight=1):
    cost = {}

    if 'AND' in condition:
        and_nodes = condition['AND']
        path_key = ' AND '.join(and_nodes)
        total_cost = sum(heuristics[node] + weight for node in and_nodes)
        cost[path_key] = total_cost

    if 'OR' in condition:
        or_nodes = condition['OR']
        path_key = ' OR '.join(or_nodes)
        total_cost = min(heuristics[node] + weight for node in or_nodes)
        cost[path_key] = total_cost

    return cost


In [2]:
# ------------------------------
# Update Heuristic Table Bottom-Up
# ------------------------------
def update_heuristics(heuristics, conditions, weight=1):
    updated_costs = {}
    nodes = list(conditions.keys())
    nodes.reverse()  # Bottom-up update

    for node in nodes:
        condition = conditions[node]
        costs = calculate_cost(heuristics, condition, weight)
        heuristics[node] = min(costs.values())  # Update heuristic table
        updated_costs[node] = calculate_cost(heuristics, condition, weight)

        print(f"{node} : {condition} => {costs}")

    return updated_costs


In [3]:
# ------------------------------
# Find the Shortest Path Recursively
# ------------------------------
def find_shortest_path(start, updated_costs, heuristics):
    path = start

    if start in updated_costs:
        min_cost = min(updated_costs[start].values())
        path_options = list(updated_costs[start].keys())
        path_values = list(updated_costs[start].values())
        index = path_values.index(min_cost)

        next_nodes = path_options[index].split()

        if len(next_nodes) == 1:
            # OR case
            next_node = next_nodes[0]
            path += " <-- " + find_shortest_path(next_node, updated_costs, heuristics)
        else:
            # AND case
            path += " <-- (" + path_options[index] + ") ["
            first_part = find_shortest_path(next_nodes[0], updated_costs, heuristics)
            second_part = find_shortest_path(next_nodes[-1], updated_costs, heuristics)
            path += first_part + " + " + second_part + "]"

    return path


In [6]:
# ------------------------------
# Example: Use the Solver
# ------------------------------
# Heuristic values for each node (initial guess)
heuristics = {
    'A': -1, 'B': 5, 'C': 2, 'D': 4,
    'E': 7, 'F': 9, 'G': 3,
    'H': 0, 'I': 0, 'J': 0
}

# Conditions for each parent node
conditions = {
    'A': {'OR': ['B'], 'AND': ['C', 'D']},
    'B': {'OR': ['E', 'F']},
    'C': {'OR': ['G'], 'AND': ['H', 'I']},
    'D': {'OR': ['J']}
}

# Step 1: Update heuristics based on conditions
print("\n--- Updating Heuristics ---")
updated_costs = update_heuristics(heuristics, conditions, weight=1)

# Step 2: Display shortest path from A
print("\n--- Shortest Path from A ---")
shortest = find_shortest_path('A', updated_costs, heuristics)
print("Shortest Path:", shortest)




--- Updating Heuristics ---
D : {'OR': ['J']} => {'J': 1}
C : {'OR': ['G'], 'AND': ['H', 'I']} => {'H AND I': 2, 'G': 4}
B : {'OR': ['E', 'F']} => {'E OR F': 8}
A : {'OR': ['B'], 'AND': ['C', 'D']} => {'C AND D': 5, 'B': 9}

--- Shortest Path from A ---
Shortest Path: A <-- (C AND D) [C <-- (H AND I) [H + I] + D <-- J]
